In [ ]:
selected_files = [
    '2405.04246v1.json',
    '2409.05034.json',
    '2402.12741.json',
    '2402.04867.json',
    '2208.13266.json',
    '2408.08673.json',
    '2404.17870v1.json',
    '2410.07753.json',
    '2408.08214.json',
    '2409.00248.json']